In [49]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium==0.5.0
import folium


In [50]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [51]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [52]:
neighborhoods_data = newyork_data['features']


In [53]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [54]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [55]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [56]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [57]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [58]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [59]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [60]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


In [61]:
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

In [62]:
CLIENT_ID = 'H5XDC1KESB0ZA41PQUXIMFDICAYWAG5044SJ0VKZXLILKWLV' # your Foursquare ID
CLIENT_SECRET = '1LYJRRW1QXMKLRFUZXHH5KTINBOW1L2FH3KYEA0AF2BWNFV3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: H5XDC1KESB0ZA41PQUXIMFDICAYWAG5044SJ0VKZXLILKWLV
CLIENT_SECRET:1LYJRRW1QXMKLRFUZXHH5KTINBOW1L2FH3KYEA0AF2BWNFV3


In [63]:
manhattan_data.loc[2, 'Neighborhood']

'Washington Heights'

In [64]:
neighborhood_latitude = manhattan_data.loc[2, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[2, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[2, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Washington Heights are 40.85190252555305, -73.93690027985234.


In [66]:

search_query = 'salon'
radius = 500
print(search_query + ' .... OK!')





salon .... OK!


In [67]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=H5XDC1KESB0ZA41PQUXIMFDICAYWAG5044SJ0VKZXLILKWLV&client_secret=1LYJRRW1QXMKLRFUZXHH5KTINBOW1L2FH3KYEA0AF2BWNFV3&ll=40.7896239,-73.9598939&v=20180605&query=salon&radius=500&limit=100'

In [65]:
cn=['Name','Location','Latitude','Longitude','Distance']
df = pd.DataFrame(columns=cn)

df

,Name,Location,Latitude,Longitude,Distance


In [68]:
results = requests.get(url).json()
results


{'meta': {'code': 200, 'requestId': '5e0742b2d03993001be5351e'},
 'response': {'venues': [{'id': '4be5e75e2457a593e531ac15',
    'name': 'Salon 94',
    'location': {'address': '12 E 94th St',
     'crossStreet': '5th Avenue',
     'lat': 40.78611385127493,
     'lng': -73.95610454732719,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.78611385127493,
       'lng': -73.95610454732719}],
     'distance': 504,
     'postalCode': '10128',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['12 E 94th St (5th Avenue)',
      'New York, NY 10128',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1e2931735',
      'name': 'Art Gallery',
      'pluralName': 'Art Galleries',
      'shortName': 'Art Gallery',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/artgallery_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1577534153',
    'h

In [69]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1e2931735', 'name': 'A...",False,4be5e75e2457a593e531ac15,12 E 94th St,US,New York,United States,5th Avenue,504,"[12 E 94th St (5th Avenue), New York, NY 10128...","[{'label': 'display', 'lat': 40.78611385127493...",40.786114,-73.956105,10128,NY,Salon 94,v-1577534153
1,"[{'id': '4bf58dd8d48988d110951735', 'name': 'S...",False,4b7997bef964a52001042fe3,Madison Avenue,US,New York,United States,96th & 97th,530,"[Madison Avenue (96th & 97th), New York, NY 10...","[{'label': 'display', 'lat': 40.787783, 'lng':...",40.787783,-73.954084,10029,NY,Irene & Abdel Hair Salon,v-1577534153
2,"[{'id': '4bf58dd8d48988d110951735', 'name': 'S...",False,50df494ce4b09e22f362262c,NaN,US,New York,United States,65th & Madison,553,"[65th & Madison, New York, NY 10128, United St...","[{'label': 'display', 'lat': 40.78681146734951...",40.786811,-73.954480,10128,NY,Pentomo Hair Salon,v-1577534153
3,"[{'id': '4bf58dd8d48988d110951735', 'name': 'S...",False,4ef34aba6da16847ce1394e3,"1409 Madison Ave, Manhattan, NY 10029",US,New York,United States,NaN,562,"[1409 Madison Ave, Manhattan, NY 10029, New Yo...","[{'label': 'display', 'lat': 40.78783352358178...",40.787834,-73.953648,10029,NY,Bodycraft Salon,v-1577534153
4,"[{'id': '4bf58dd8d48988d110951735', 'name': 'S...",False,4bb8dfccb35776b0b456c901,50 E 96th St,US,New York,United States,Madison Ave & 96th Street,562,"[50 E 96th St (Madison Ave & 96th Street), New...","[{'label': 'display', 'lat': 40.787167, 'lng':...",40.787167,-73.954056,10128,NY,Prestige Hair Salon,v-1577534153


In [71]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Salon 94,Art Gallery,12 E 94th St,US,New York,United States,5th Avenue,504,"[12 E 94th St (5th Avenue), New York, NY 10128...","[{'label': 'display', 'lat': 40.78611385127493...",40.786114,-73.956105,10128,NY,4be5e75e2457a593e531ac15
1,Irene & Abdel Hair Salon,Salon / Barbershop,Madison Avenue,US,New York,United States,96th & 97th,530,"[Madison Avenue (96th & 97th), New York, NY 10...","[{'label': 'display', 'lat': 40.787783, 'lng':...",40.787783,-73.954084,10029,NY,4b7997bef964a52001042fe3
2,Pentomo Hair Salon,Salon / Barbershop,NaN,US,New York,United States,65th & Madison,553,"[65th & Madison, New York, NY 10128, United St...","[{'label': 'display', 'lat': 40.78681146734951...",40.786811,-73.954480,10128,NY,50df494ce4b09e22f362262c
3,Bodycraft Salon,Salon / Barbershop,"1409 Madison Ave, Manhattan, NY 10029",US,New York,United States,NaN,562,"[1409 Madison Ave, Manhattan, NY 10029, New Yo...","[{'label': 'display', 'lat': 40.78783352358178...",40.787834,-73.953648,10029,NY,4ef34aba6da16847ce1394e3
4,Prestige Hair Salon,Salon / Barbershop,50 E 96th St,US,New York,United States,Madison Ave & 96th Street,562,"[50 E 96th St (Madison Ave & 96th Street), New...","[{'label': 'display', 'lat': 40.787167, 'lng':...",40.787167,-73.954056,10128,NY,4bb8dfccb35776b0b456c901


In [72]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map